# Prediction of the performance of students in a particular student based on the subject code(taking last 2 digits into considration) ,Output = GRADE .....Using ML libraries

### importing required modules

In [1]:
from pyspark.sql import SparkSession
import sys
import os
import matplotlib.pyplot as plt
import pandas as pd
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import OneHotEncoder

### creating Spark session

In [2]:
spark = SparkSession.builder.appName('Practise').getOrCreate()

22/04/14 18:06:12 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.0.195 instead (on interface wlp0s20f3)
22/04/14 18:06:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/14 18:06:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/04/14 18:06:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
spark

### reading csv file

In [4]:
df = spark.read.csv('Results_Data.csv',header=True,inferSchema=True)
df.show()

+----------+--------+---+---+---+------+-----+-----+--------+
|       USN|   SCODE|CIE|SEE|TOT|RESULT|GRADE| DATE|ExamType|
+----------+--------+---+---+---+------+-----+-----+--------+
|4JN18CS001| 18MAT11| 32| 41| 73|     P|    8|12019| Regular|
|4JN18CS001| 18PHY12| 29| 47| 76|     P|    8|12019| Regular|
|4JN18CS001| 18ELE13| 25| 43| 68|     P|    7|12019| Regular|
|4JN18CS001| 18CIV14| 31| 26| 57|     P|    6|12019| Regular|
|4JN18CS001|18EGDL15| 31| 56| 87|     P|    9|12019| Regular|
|4JN18CS001|18PHYL16| 32| 40| 72|     P|    8|12019| Regular|
|4JN18CS001|18ELEL17| 34| 52| 86|     P|    9|12019| Regular|
|4JN18CS001| 18EGH18| 27| 32| 59|     P|    6|12019| Regular|
|4JN18CS002| 18MAT11| 33| 51| 84|     P|    9|12019| Regular|
|4JN18CS002| 18PHY12| 36| 45| 81|     P|    9|12019| Regular|
|4JN18CS002| 18ELE13| 37| 31| 68|     P|    7|12019| Regular|
|4JN18CS002| 18CIV14| 34| 29| 63|     P|    7|12019| Regular|
|4JN18CS002|18EGDL15| 34| 55| 89|     P|    9|12019| Regular|
|4JN18CS

### preprosessnig dataframe

In [5]:
df.createOrReplaceTempView("df2_view")

dffinal = spark.sql(
    """
    SELECT USN,GRADE,
    (CASE
    WHEN SCODE LIKE '%1' THEN 1
    WHEN SCODE LIKE '%2' THEN 2
    WHEN SCODE LIKE '%3' THEN 3
    WHEN SCODE LIKE '%4' THEN 4
    WHEN SCODE LIKE '%5' THEN 5
    WHEN SCODE LIKE '%6' THEN 6
    WHEN SCODE LIKE '%7' THEN 7
    WHEN SCODE LIKE '%8' THEN 8
    WHEN SCODE LIKE '%9' THEN 9
    END) AS SCODEFINAL,
    (CASE
    WHEN DATE LIKE '%19' THEN 19
    WHEN DATE LIKE '%20' THEN 20
    END) AS NEWDATE
    
    FROM df2_view
    """
)

dffinal.show()

+----------+-----+----------+-------+
|       USN|GRADE|SCODEFINAL|NEWDATE|
+----------+-----+----------+-------+
|4JN18CS001|    8|         1|     19|
|4JN18CS001|    8|         2|     19|
|4JN18CS001|    7|         3|     19|
|4JN18CS001|    6|         4|     19|
|4JN18CS001|    9|         5|     19|
|4JN18CS001|    8|         6|     19|
|4JN18CS001|    9|         7|     19|
|4JN18CS001|    6|         8|     19|
|4JN18CS002|    9|         1|     19|
|4JN18CS002|    9|         2|     19|
|4JN18CS002|    7|         3|     19|
|4JN18CS002|    7|         4|     19|
|4JN18CS002|    9|         5|     19|
|4JN18CS002|    8|         6|     19|
|4JN18CS002|    9|         7|     19|
|4JN18CS002|    8|         8|     19|
|4JN18CS003|    9|         1|     19|
|4JN18CS003|    8|         2|     19|
|4JN18CS003|    9|         3|     19|
|4JN18CS003|    8|         4|     19|
+----------+-----+----------+-------+
only showing top 20 rows



In [6]:
qualification_indexer = StringIndexer(inputCol="USN", outputCol="EncodedUSN")
#Fits a model to the input dataset with optional parameters.
df1 = qualification_indexer.fit(dffinal).transform(dffinal)
df1.show()

+----------+-----+----------+-------+----------+
|       USN|GRADE|SCODEFINAL|NEWDATE|EncodedUSN|
+----------+-----+----------+-------+----------+
|4JN18CS001|    8|         1|     19|     242.0|
|4JN18CS001|    8|         2|     19|     242.0|
|4JN18CS001|    7|         3|     19|     242.0|
|4JN18CS001|    6|         4|     19|     242.0|
|4JN18CS001|    9|         5|     19|     242.0|
|4JN18CS001|    8|         6|     19|     242.0|
|4JN18CS001|    9|         7|     19|     242.0|
|4JN18CS001|    6|         8|     19|     242.0|
|4JN18CS002|    9|         1|     19|     243.0|
|4JN18CS002|    9|         2|     19|     243.0|
|4JN18CS002|    7|         3|     19|     243.0|
|4JN18CS002|    7|         4|     19|     243.0|
|4JN18CS002|    9|         5|     19|     243.0|
|4JN18CS002|    8|         6|     19|     243.0|
|4JN18CS002|    9|         7|     19|     243.0|
|4JN18CS002|    8|         8|     19|     243.0|
|4JN18CS003|    9|         1|     19|     244.0|
|4JN18CS003|    8|  

In [7]:
dffinal.printSchema

<bound method DataFrame.printSchema of DataFrame[USN: string, GRADE: int, SCODEFINAL: int, NEWDATE: int]>

In [8]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=['EncodedUSN','NEWDATE','SCODEFINAL'],outputCol="NEW")
output=featureassembler.transform(df1)
output.show()

+----------+-----+----------+-------+----------+----------------+
|       USN|GRADE|SCODEFINAL|NEWDATE|EncodedUSN|             NEW|
+----------+-----+----------+-------+----------+----------------+
|4JN18CS001|    8|         1|     19|     242.0|[242.0,19.0,1.0]|
|4JN18CS001|    8|         2|     19|     242.0|[242.0,19.0,2.0]|
|4JN18CS001|    7|         3|     19|     242.0|[242.0,19.0,3.0]|
|4JN18CS001|    6|         4|     19|     242.0|[242.0,19.0,4.0]|
|4JN18CS001|    9|         5|     19|     242.0|[242.0,19.0,5.0]|
|4JN18CS001|    8|         6|     19|     242.0|[242.0,19.0,6.0]|
|4JN18CS001|    9|         7|     19|     242.0|[242.0,19.0,7.0]|
|4JN18CS001|    6|         8|     19|     242.0|[242.0,19.0,8.0]|
|4JN18CS002|    9|         1|     19|     243.0|[243.0,19.0,1.0]|
|4JN18CS002|    9|         2|     19|     243.0|[243.0,19.0,2.0]|
|4JN18CS002|    7|         3|     19|     243.0|[243.0,19.0,3.0]|
|4JN18CS002|    7|         4|     19|     243.0|[243.0,19.0,4.0]|
|4JN18CS00

In [9]:
df2 = output.limit(10000)
df2.show()

+----------+-----+----------+-------+----------+----------------+
|       USN|GRADE|SCODEFINAL|NEWDATE|EncodedUSN|             NEW|
+----------+-----+----------+-------+----------+----------------+
|4JN18CS001|    8|         1|     19|     242.0|[242.0,19.0,1.0]|
|4JN18CS001|    8|         2|     19|     242.0|[242.0,19.0,2.0]|
|4JN18CS001|    7|         3|     19|     242.0|[242.0,19.0,3.0]|
|4JN18CS001|    6|         4|     19|     242.0|[242.0,19.0,4.0]|
|4JN18CS001|    9|         5|     19|     242.0|[242.0,19.0,5.0]|
|4JN18CS001|    8|         6|     19|     242.0|[242.0,19.0,6.0]|
|4JN18CS001|    9|         7|     19|     242.0|[242.0,19.0,7.0]|
|4JN18CS001|    6|         8|     19|     242.0|[242.0,19.0,8.0]|
|4JN18CS002|    9|         1|     19|     243.0|[243.0,19.0,1.0]|
|4JN18CS002|    9|         2|     19|     243.0|[243.0,19.0,2.0]|
|4JN18CS002|    7|         3|     19|     243.0|[243.0,19.0,3.0]|
|4JN18CS002|    7|         4|     19|     243.0|[243.0,19.0,4.0]|
|4JN18CS00

In [10]:
finalized_data = df2.select(['''NEW','GRADE'])
finalized_data.show()

+----------------+-----+
|             NEW|GRADE|
+----------------+-----+
|[242.0,19.0,1.0]|    8|
|[242.0,19.0,2.0]|    8|
|[242.0,19.0,3.0]|    7|
|[242.0,19.0,4.0]|    6|
|[242.0,19.0,5.0]|    9|
|[242.0,19.0,6.0]|    8|
|[242.0,19.0,7.0]|    9|
|[242.0,19.0,8.0]|    6|
|[243.0,19.0,1.0]|    9|
|[243.0,19.0,2.0]|    9|
|[243.0,19.0,3.0]|    7|
|[243.0,19.0,4.0]|    7|
|[243.0,19.0,5.0]|    9|
|[243.0,19.0,6.0]|    8|
|[243.0,19.0,7.0]|    9|
|[243.0,19.0,8.0]|    8|
|[244.0,19.0,1.0]|    9|
|[244.0,19.0,2.0]|    8|
|[244.0,19.0,3.0]|    9|
|[244.0,19.0,4.0]|    8|
+----------------+-----+
only showing top 20 rows



### ML training starts 

In [11]:
train, test = finalized_data.randomSplit([0.9, 0.1])
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 8971
Test Dataset Count: 1029


In [12]:
rf = RandomForestClassifier(featuresCol = 'NEW', labelCol = 'GRADE',maxBins=720)
rfModel = rf.fit(train)

In [13]:
predictions = rfModel.transform(test)
predictions.select(['USN','NEW','GRADE','prediction']).show(25)

AnalysisException: cannot resolve 'USN' given input columns: [df2_view.GRADE, NEW, prediction, probability, rawPrediction];
'Project ['USN, NEW#174, GRADE#22, prediction#370]
+- Project [NEW#174, GRADE#22, rawPrediction#352, probability#359, UDF(rawPrediction#352) AS prediction#370]
   +- Project [NEW#174, GRADE#22, rawPrediction#352, UDF(rawPrediction#352) AS probability#359]
      +- Project [NEW#174, GRADE#22, UDF(NEW#174) AS rawPrediction#352]
         +- Sample 0.9, 1.0, false, 2919333773569388529
            +- Sort [NEW#174 ASC NULLS FIRST, GRADE#22 ASC NULLS FIRST], false
               +- Project [NEW#174, GRADE#22]
                  +- GlobalLimit 10000
                     +- LocalLimit 10000
                        +- Project [USN#16, GRADE#22, SCODEFINAL#80, NEWDATE#81, EncodedUSN#137, UDF(struct(EncodedUSN, EncodedUSN#137, NEWDATE_double_VectorAssembler_28fc142d4bcc, cast(NEWDATE#81 as double), SCODEFINAL_double_VectorAssembler_28fc142d4bcc, cast(SCODEFINAL#80 as double))

In [ ]:
predictions.select("GRADE", "prediction").show(10)

In [ ]:
dftest = spark.read.csv('new.csv',header=True,inferSchema=True)
dftest.show()
dftest = dftest.limit(15000)

In [ ]:
qualification_indexer = StringIndexer(inputCol="USN", outputCol="EncodedUSN")
#Fits a model to the input dataset with optional parameters.
dftest1 = qualification_indexer.fit(dftest).transform(dftest)
dftest1.show()

In [ ]:
dftest1.createOrReplaceTempView("df3_view")

dffinal2 = spark.sql(
    """
    SELECT EncodedUSN,GRADE,
    (CASE
    WHEN SCODE LIKE '%1' THEN 1
    WHEN SCODE LIKE '%2' THEN 2
    WHEN SCODE LIKE '%3' THEN 3
    WHEN SCODE LIKE '%4' THEN 4
    WHEN SCODE LIKE '%5' THEN 5
    WHEN SCODE LIKE '%6' THEN 6
    WHEN SCODE LIKE '%7' THEN 7
    WHEN SCODE LIKE '%8' THEN 8
    WHEN SCODE LIKE '%9' THEN 9
    END) AS SCODEFINAL,
    (CASE
    WHEN 2021 LIKE '%21' THEN 21
    END) AS NEWDATE
    
    FROM df3_view
    """
)

dffinal2.show()

In [ ]:
featureassembler=VectorAssembler(inputCols=['EncodedUSN','NEWDATE','SCODEFINAL'],outputCol="NEW")
output1=featureassembler.transform(dffinal2)
output1.show()

In [ ]:
testfinal = output1.select('NEW','GRADE')

In [ ]:
df1.select('EncodedUSN').show(50)

In [ ]:
StringIndexer(inputCols=["USN"], outputCols=["newUSN1"])
encoder = OneHotEncoder(
    inputCols=["newUSN1"],  
    outputCols=["newUSN"]
)

In [ ]:
indexer = StringIndexer(inputCol="USN", outputCol="newUSN").fit(dffinal)
indexed_df = indexer.transform(dffinal)
indexed_df.show()

### Prediction

In [ ]:
predictions = rfModel.transform(testfinal)
predictions.select(['NEW','GRADE','prediction']).show(25)

In [ ]:
mPath = "Model1"
rfModel.write().overwrite().save(mPath)